<a href="https://colab.research.google.com/github/AwaisEhsan67/Kyber512-and-NTRU509-comparison/blob/main/PQClean_Library_for_Kyber_512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!sudo apt-get update
!sudo apt-get install build-essential git


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [5]:
!git clone https://github.com/PQClean/PQClean.git


Cloning into 'PQClean'...
remote: Enumerating objects: 33664, done.
remote: Counting objects: 100% (3106/3106), done.
remote: Compressing objects: 100% (728/728), done.
remote: Total 33664 (delta 2772), reused 2378 (delta 2378), pack-reused 30558 (from 3)
Receiving objects: 100% (33664/33664), 7.44 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (26180/26180), done.


In [14]:
!cd PQClean/crypto_kem/ml-kem-512/clean/


In [15]:
code = """
#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>
#include <string.h>
#include "api.h"

int main() {
    uint8_t pk[PQCLEAN_MLKEM512_CLEAN_CRYPTO_PUBLICKEYBYTES];
    uint8_t sk[PQCLEAN_MLKEM512_CLEAN_CRYPTO_SECRETKEYBYTES];
    uint8_t ct[PQCLEAN_MLKEM512_CLEAN_CRYPTO_CIPHERTEXTBYTES];
    uint8_t key_a[PQCLEAN_MLKEM512_CLEAN_CRYPTO_BYTES];
    uint8_t key_b[PQCLEAN_MLKEM512_CLEAN_CRYPTO_BYTES];

    PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair(pk, sk);
    PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc(ct, key_a, pk);
    PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec(key_b, ct, sk);

    printf("Shared keys match: %s\\n", memcmp(key_a, key_b, PQCLEAN_MLKEM512_CLEAN_CRYPTO_BYTES) == 0 ? "YES" : "NO");

    return 0;
}
"""
with open("test_kyber.c", "w") as f:
    f.write(code)


In [16]:
!gcc -o test_kyber test_kyber.c \
    kem.c indcpa.c verify.c cbd.c ntt.c reduce.c polyvec.c poly.c symmetric-shake.c \
    ../../../common/randombytes.c \
    ../../../common/fips202.c \
    -I . -I ../../../common


test_kyber.c:6:10: fatal error: api.h: No such file or directory
    6 | #include "api.h"
      |          ^~~~~~~
compilation terminated.
cc1: fatal error: kem.c: No such file or directory
compilation terminated.
cc1: fatal error: indcpa.c: No such file or directory
compilation terminated.
cc1: fatal error: verify.c: No such file or directory
compilation terminated.
cc1: fatal error: cbd.c: No such file or directory
compilation terminated.
cc1: fatal error: ntt.c: No such file or directory
compilation terminated.
cc1: fatal error: reduce.c: No such file or directory
compilation terminated.
cc1: fatal error: polyvec.c: No such file or directory
compilation terminated.
cc1: fatal error: poly.c: No such file or directory
compilation terminated.
cc1: fatal error: symmetric-shake.c: No such file or directory
compilation terminated.
cc1: fatal error: ../../../common/randombytes.c: No such file or directory
compilation terminated.
cc1: fatal error: ../../../common/fips202.c: No such file or 

In [9]:
!./test_kyber


/bin/bash: line 1: ./test_kyber: No such file or directory


In [12]:
!gcc -shared -fPIC \
    kem.c indcpa.c verify.c cbd.c ntt.c reduce.c polyvec.c poly.c symmetric-shake.c \
    ../../../common/randombytes.c ../../../common/fips202.c \
    -I . -I ../../../common \
    -o libkyber512.so


cc1: fatal error: kem.c: No such file or directory
compilation terminated.
cc1: fatal error: indcpa.c: No such file or directory
compilation terminated.
cc1: fatal error: verify.c: No such file or directory
compilation terminated.
cc1: fatal error: cbd.c: No such file or directory
compilation terminated.
cc1: fatal error: ntt.c: No such file or directory
compilation terminated.
cc1: fatal error: reduce.c: No such file or directory
compilation terminated.
cc1: fatal error: polyvec.c: No such file or directory
compilation terminated.
cc1: fatal error: poly.c: No such file or directory
compilation terminated.
cc1: fatal error: symmetric-shake.c: No such file or directory
compilation terminated.
cc1: fatal error: ../../../common/randombytes.c: No such file or directory
compilation terminated.
cc1: fatal error: ../../../common/fips202.c: No such file or directory
compilation terminated.


In [11]:
import ctypes
import os


lib = ctypes.CDLL("./libkyber512.so")


pk = ctypes.create_string_buffer(800)
sk = ctypes.create_string_buffer(1632)
ct = ctypes.create_string_buffer(768)
ss = ctypes.create_string_buffer(32)


lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]


lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair(pk, sk)
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc(ct, ss, pk)
ss_dec = ctypes.create_string_buffer(32)
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec(ss_dec, ct, sk)


print("Shared secret match:", ss.raw == ss_dec.raw)


OSError: ./libkyber512.so: cannot open shared object file: No such file or directory

In [9]:
import ctypes
import time
import statistics


lib = ctypes.CDLL("./libkyber512.so")


lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]


CRYPTO_PUBLICKEYBYTES = 800
CRYPTO_SECRETKEYBYTES = 1632


trials = 1000
times = []

for _ in range(trials):
    pk = ctypes.create_string_buffer(CRYPTO_PUBLICKEYBYTES)
    sk = ctypes.create_string_buffer(CRYPTO_SECRETKEYBYTES)

    start = time.perf_counter()
    lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair(pk, sk)
    end = time.perf_counter()

    times.append((end - start) * 1000)  # ms


print(f"Ran {trials} trials of Kyber-512 key generation")
print(f"Average time: {statistics.mean(times):.3f} ms")
print(f"Min time: {min(times):.3f} ms")
print(f"Max time: {max(times):.3f} ms")
print(f"Standard deviation: {statistics.stdev(times):.3f} ms")


Ran 1000 trials of Kyber-512 key generation
Average time: 0.207 ms
Min time: 0.197 ms
Max time: 0.389 ms
Standard deviation: 0.016 ms


In [10]:
import ctypes
import time
import statistics

lib = ctypes.CDLL("./libkyber512.so")

KYBER_PUBLICKEYBYTES  = 800
KYBER_SECRETKEYBYTES  = 1632
KYBER_CIPHERTEXTBYTES = 768
KYBER_SSBYTES         = 32


keypair = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
keypair.restype  = ctypes.c_int

encapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc
encapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
encapsulate.restype  = ctypes.c_int

decapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec
decapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
decapsulate.restype  = ctypes.c_int

def functional_test():
    pk = ctypes.create_string_buffer(KYBER_PUBLICKEYBYTES)
    sk = ctypes.create_string_buffer(KYBER_SECRETKEYBYTES)
    ct = ctypes.create_string_buffer(KYBER_CIPHERTEXTBYTES)
    ss = ctypes.create_string_buffer(KYBER_SSBYTES)
    ss_dec = ctypes.create_string_buffer(KYBER_SSBYTES)

    assert keypair(pk, sk) == 0, "Keypair failed"
    assert encapsulate(ct, ss, pk) == 0, "Encapsulation failed"
    assert decapsulate(ss_dec, ct, sk) == 0, "Decapsulation failed"

    match = ss.raw == ss_dec.raw
    print(f"[Functional] Shared secret match: {match}")

def benchmark(fn, buf_sizes, trials=1000):
    """Run fn(bufs...) for trials and return list of ms timings."""
    times = []
    for _ in range(trials):
        bufs = [ctypes.create_string_buffer(sz) for sz in buf_sizes]
        start = time.perf_counter()
        assert fn(*bufs) == 0
        end = time.perf_counter()
        times.append((end - start) * 1000)
    return times

def print_stats(name, times):
    print(f"[Benchmark] {name}:")
    print(f"  avg = {statistics.mean(times):.3f} ms")
    print(f"  min = {min(times):.3f} ms")
    print(f"  max = {max(times):.3f} ms")
    print(f"  std = {statistics.stdev(times):.3f} ms")


if __name__ == "__main__":
    # 1) Functional correctness
    functional_test()

    # 2) Benchmarks
    trials = 1000
    # Key generation
    t_keygen = benchmark(keypair, [KYBER_PUBLICKEYBYTES, KYBER_SECRETKEYBYTES], trials)
    print_stats("Kyber KeyGen", t_keygen)

    # Encapsulation
    t_enc = benchmark(encapsulate,
                      [KYBER_CIPHERTEXTBYTES, KYBER_SSBYTES, KYBER_PUBLICKEYBYTES],
                      trials)
    print_stats("Kyber Encapsulate", t_enc)

    # Decapsulation
    t_dec = benchmark(decapsulate,
                      [KYBER_SSBYTES, KYBER_CIPHERTEXTBYTES, KYBER_SECRETKEYBYTES],
                      trials)
    print_stats("Kyber Decapsulate", t_dec)


[Functional] Shared secret match: True
[Benchmark] Kyber KeyGen:
  avg = 0.211 ms
  min = 0.197 ms
  max = 0.760 ms
  std = 0.033 ms
[Benchmark] Kyber Encapsulate:
  avg = 0.264 ms
  min = 0.247 ms
  max = 0.822 ms
  std = 0.029 ms
[Benchmark] Kyber Decapsulate:
  avg = 0.357 ms
  min = 0.323 ms
  max = 1.688 ms
  std = 0.067 ms


In [13]:
!pip install psutil


In [14]:
import ctypes
import psutil
import time


lib = ctypes.CDLL("./libkyber512.so")

keypair = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
keypair.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
keypair.restype  = ctypes.c_int


KYBER_PK = 800
KYBER_SK = 1632


proc = psutil.Process()


pk = ctypes.create_string_buffer(KYBER_PK)
sk = ctypes.create_string_buffer(KYBER_SK)
keypair(pk, sk)

# Force a short sleep to let OS settle memory
time.sleep(0.1)


mem_before = proc.memory_info().rss


pk = ctypes.create_string_buffer(KYBER_PK)
sk = ctypes.create_string_buffer(KYBER_SK)
keypair(pk, sk)

mem_after = proc.memory_info().rss

print(f"Kyber-512 KeyGen Memory (RSS):")
print(f"  Before: {mem_before/1024:.1f} KB")
print(f"  After:  {mem_after/1024:.1f} KB")
print(f"  Delta:  {(mem_after-mem_before)/1024:.1f} KB")


Kyber-512 KeyGen Memory (RSS):
  Before: 289248.0 KB
  After:  289248.0 KB
  Delta:  0.0 KB


In [16]:

%%shell
/usr/bin/time -f "Kyber-512 KeyGen PeakRSS: %M KB" python3 - << 'PYCODE'
import ctypes

lib = ctypes.CDLL("./libkyber512.so")
kp  = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
kp.argtypes = [ctypes.c_void_p, ctypes.c_void_p]

pk = ctypes.create_string_buffer(800)
sk = ctypes.create_string_buffer(1632)
kp(pk, sk)
PYCODE


Kyber-512 KeyGen PeakRSS: 13580 KB


In [18]:
import ctypes
import time

kyber = ctypes.CDLL("./libkyber512.so")
keypair = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
keypair.restype  = ctypes.c_int

encapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc
encapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
encapsulate.restype  = ctypes.c_int

decapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec
decapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
decapsulate.restype  = ctypes.c_int


pk = ctypes.create_string_buffer(800)
sk = ctypes.create_string_buffer(1600)
ct = ctypes.create_string_buffer(768)
ss = ctypes.create_string_buffer(32)

start = time.perf_counter_ns()
keypair(pk, sk)
keygen_time = time.perf_counter_ns() - start

start = time.perf_counter_ns()
encapsulate(ct, ss, pk)
enc_time = time.perf_counter_ns() - start

start = time.perf_counter_ns()
decapsulate(ss, ct, sk)
dec_time = time.perf_counter_ns() - start


keygen_time //= 1000
enc_time    //= 1000
dec_time    //= 1000

print("KeyGen:", keygen_time, "µs")
print("Encapsulation:", enc_time, "µs")
print("Decapsulation:", dec_time, "µs")


KeyGen: 1596 µs
Encapsulation: 546 µs
Decapsulation: 686 µs


In [20]:
!lscpu


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [21]:
!free -h


               total        used        free      shared  buff/cache   available
Mem:            12Gi       870Mi       8.1Gi       2.0Mi       3.7Gi        11Gi
Swap:             0B          0B          0B


In [19]:
!cat /proc/cpuinfo
!cat /proc/meminfo


processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
ad

In [3]:
!pip install pyRAPL


In [4]:

# following code will work on laptop and Raspberry Pi only, not on collab.
import ctypes
import pyRAPL
import sys


pyRAPL.setup()
meter = pyRAPL.Measurement('kyber_full_kem')


lib = ctypes.CDLL("./libkyber512.so")
kp = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
enc = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc
dec = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec

kp.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
enc.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
dec.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]


PK_BYTES = 800
SK_BYTES = 1632
CT_BYTES = 768
SS_BYTES = 32

pk = ctypes.create_string_buffer(PK_BYTES)
sk = ctypes.create_string_buffer(SK_BYTES)
ct = ctypes.create_string_buffer(CT_BYTES)
ss = ctypes.create_string_buffer(SS_BYTES)


meter.begin()
kp(pk, sk)
enc(ct, ss, pk)
dec(ss, ct, sk)
meter.end()

print("Kyber-512 full KEM energy (µJ):", meter.result.pkg)


PyRAPLCantRecordEnergyConsumption: 

KeyboardInterrupt: 

In [3]:
import time

bandwidth_mbps = 10  # Mbps
bandwidth_bps = bandwidth_mbps * 1_000_000

kyber_total_bytes = 800 + 768  # pk + ct
ntru_total_bytes = 699 + 699   # pk + ct

def simulate_network_transfer(data_bytes):
    data_bits = data_bytes * 8
    expected_time = data_bits / bandwidth_bps  # seconds
    start = time.time()
    time.sleep(expected_time)
    end = time.time()
    return end - start


kyber_time = simulate_network_transfer(kyber_total_bytes)
print(f"Kyber512 simulated transmission time: {kyber_time * 1000:.2f} ms")


ntru_time = simulate_network_transfer(ntru_total_bytes)
print(f"NTRU509 simulated transmission time: {ntru_time * 1000:.2f} ms")


Kyber512 simulated transmission time: 1.54 ms
NTRU509 simulated transmission time: 1.21 ms


In [17]:
#additional code, will try to run in future for further analysis
import re
import os


api_path = os.path.join("PQClean","crypto_kem","ml-kem-512","clean","api.h")
with open(api_path, "r") as f:
    hdr = f.read()

pk_bytes = int(re.search(r"#define\s+CRYPTO_PUBLICKEYBYTES\s+(\d+)", hdr).group(1))
ct_bytes = int(re.search(r"#define\s+CRYPTO_CIPHERTEXTBYTES\s+(\d+)", hdr).group(1))


total_bytes = pk_bytes + ct_bytes
bandwidth_bps = 10_000_000  # 10 Mbps
trans_time_ms = (total_bytes * 8) / bandwidth_bps * 1000

print("Kyber-512 Bandwidth Metrics")
print(f"  Public key size     : {pk_bytes} bytes")
print(f"  Ciphertext size     : {ct_bytes} bytes")
print(f"  Total transmission  : {total_bytes} bytes")
print(f"  Est. trans. delay   : {trans_time_ms:.2f} ms")


AttributeError: 'NoneType' object has no attribute 'group'